In [ ]:
from requests_html import HTMLSession
import json
import os

with open('Issuelist/jf_issuelist.json','r') as f:
    jf = f.read()
JF_ISSUELIST = json.loads(jf)

### TEST

In [ ]:
test_url = 'https://onlinelibrary.wiley.com/toc/15406261/2019/74/2'
s = HTMLSession()
r = s.get(test_url)

In [ ]:
# vol iss
vol_iss = r.html.find('div.cover-image__parent-item',first=True).text
vol = vol_iss.split(' ')[1].strip(',')
iss = vol_iss.split('Issue ')[-1]
print(vol)
print(iss)

In [ ]:
# date
r.html.find('div.cover-image__date',first=True).text

In [ ]:
# page
r.html.find('div.cover-image__pageRange',first=True).text.replace(u'\xa0', u' ')

In [ ]:
# url_list
url_list = [i.absolute_links.pop() for i in r.html.find('div.issue-item>a.issue-item__title')]
url_list

In [ ]:
r = s.get('https://onlinelibrary.wiley.com/doi/10.1111/jofi.12749')

In [ ]:

temp = r.html.find('div.accordion-tabbed a.author-name>span')
for i in temp:
    print(i.text)

In [ ]:
r.html.find('div.article-section__content.en.main',first=True).text

### MAIN

In [ ]:
JF_URL = 'https://onlinelibrary.wiley.com/toc/15406261'
MAX_RETRY = 5

def get_jf(vol, iss):
    s = HTMLSession()

    #issue request
    try:
        r = s.get('{}/{}/{}/{}'.format(JF_URL,int(vol)+1945,vol,iss))
    except:
        for c , _ in enumerate(range(MAX_RETRY+1)):
            print('Retry#{} '.format(c+1), end='')
            r = s.get('{}/{}/{}/{}'.format(JF_URL,int(vol)+1945,vol,iss))
            if r.status_code == 200:
                break
    print('Volume \033[32m{}\033[m, Issue \033[32m{}\033[m: Total {} '.format(vol,iss,len(r.html.find('div.issue-item'))))
    # get issuelist
    url_list = [i.absolute_links.pop() for i in r.html.find('div.issue-item>a.issue-item__title')]
    
    # issue info
    content = {}
    vol_iss = r.html.find('div.cover-image__parent-item',first=True).text
    content['journal'] = 'JF'
    content['volume'] = vol_iss.split(' ')[1].strip(',')
    content['issue'] = vol_iss.split('Issue ')[-1]
    content['date'] = r.html.find('div.cover-image__date',first=True).text
    content['page'] = r.html.find('div.cover-image__pageRange',first=True).text.replace(u'\xa0', u' ')
    
    # article loop
    content['article'] = []
    for cnt, url in enumerate(url_list):
        print('#{} '.format(cnt+1), end='')

        
        # article request
        try:
            r = s.get(url)
        except:
            for c , _ in enumerate(range(MAX_RETRY+1)):
                print('Retry#{} '.format(c+1), end='')
                r = s.get(url)
                if r.status_code == 200:
                    break
            
        content['article'].append({})
        content['article'][cnt]['no'] = str(cnt+1)
        content['article'][cnt]['title'] = r.html.find('h2.citation__title',first=True).text
        content['article'][cnt]['date'] = r.html.find('span.epub-date',first=True).text
        # authors
        authors = [i.text for i in r.html.find('div.accordion-tabbed a.author-name>span')]
        content['article'][cnt]['author'] = authors
        # abstract
        abstract = r.html.find('div.article-section__content',first=True)
        if abstract:
            content['article'][cnt]['abstract'] = abstract.text
        else:
            content['article'][cnt]['abstract'] = ''
            
        content['article'][cnt]['link'] = r.html.find('a.epub-doi',first=True).absolute_links.pop()
    
    
    # write json file
    path_name = 'JSON/JF/Volume {}/Volume {} - Issue {}.json'.format(vol,vol,iss) 
    os.makedirs(os.path.dirname(path_name), exist_ok=True)
    with open(path_name, 'w') as f:
        # get article items
        f.write(json.dumps(content, indent=4))
    print('\033[32m{}\033[m'.format('DONE!'))
    
    
def jf_main(vol,iss):
    if os.getcwd()[13:] != 'light-speed-engine':
        print('Not in the right folder')
    else:
        get_jf(vol,iss)
    

In [ ]:
%%time
jf_main('25','3')

In [ ]:
%%time
vol = '34'
for i in JF_ISSUELIST[vol]:
    jf_main(vol,i)

In [ ]:
%%time
for i in range(1,25):
    vol = str(i)
    for j in JF_ISSUELIST[vol]:
        jf_main(vol,j)